<a href="https://colab.research.google.com/github/qvu0401/FNAF-CV/blob/main/FNAF_CV_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip -q install huggingface_hub hf_transfer torch torchvision torchaudio pillow tqdm torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 46.5 MB/s eta 0:00:00


In [9]:
import os
from huggingface_hub import hf_hub_download

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"  # speeds up downloads in Colab

DATASET_ID = "cj-mills/hagrid-sample-500k-384p"
ZIP_NAME = "hagrid-sample-500k-384p.zip"

zip_path = hf_hub_download(
    repo_id=DATASET_ID,
    filename=ZIP_NAME,
    repo_type="dataset",
    local_dir="/content/hagrid_zip",
    local_dir_use_symlinks=False,
)
print("Downloaded to:", zip_path)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_validators.py:202: UserWarning: The `local_dir_use_symlinks` argument is deprecated and ignored in `hf_hub_download`. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


hagrid-sample-500k-384p.zip:   0%|          | 0.00/13.4G [00:00<?, ?B/s]

Downloaded to: /content/hagrid_zip/hagrid-sample-500k-384p.zip


In [10]:
!mkdir -p /content/hagrid_500k
!unzip -q "/content/hagrid_zip/hagrid-sample-500k-384p.zip" -d /content/hagrid_500k
!ls -lah /content/hagrid_500k | head

total 12K
drwxr-xr-x 3 root root 4.0K Feb  6 03:55 .
drwxr-xr-x 1 root root 4.0K Feb  6 03:55 ..
drwxrwxrwx 4 root root 4.0K Jun 20  2023 hagrid-sample-500k-384p


In [11]:
import os

root = "/content/hagrid_500k"
for path, dirs, files in os.walk(root):
    print("DIR:", path)
    print("  subdirs:", dirs[:5])
    print("  files:", files[:10])
    break

DIR: /content/hagrid_500k
  subdirs: ['hagrid-sample-500k-384p']
  files: []


In [12]:
import torch, torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights

CLASSES = ["palm", "fist", "stop", "stop_inverted"]  # + "grip" only if found
class_to_idx = {c:i for i,c in enumerate(CLASSES)}

model = resnet18(weights=ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, len(CLASSES))
model = model.cuda()

loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 210MB/s]


In [13]:
!find /content/hagrid_500k -maxdepth 3 -type f | sed -n '1,120p'

/content/hagrid_500k/hagrid-sample-500k-384p/ann_train_val/dislike.json
/content/hagrid_500k/hagrid-sample-500k-384p/ann_train_val/stop.json
/content/hagrid_500k/hagrid-sample-500k-384p/ann_train_val/four.json
/content/hagrid_500k/hagrid-sample-500k-384p/ann_train_val/two_up.json
/content/hagrid_500k/hagrid-sample-500k-384p/ann_train_val/call.json
/content/hagrid_500k/hagrid-sample-500k-384p/ann_train_val/ok.json
/content/hagrid_500k/hagrid-sample-500k-384p/ann_train_val/stop_inverted.json
/content/hagrid_500k/hagrid-sample-500k-384p/ann_train_val/three2.json
/content/hagrid_500k/hagrid-sample-500k-384p/ann_train_val/peace.json
/content/hagrid_500k/hagrid-sample-500k-384p/ann_train_val/fist.json
/content/hagrid_500k/hagrid-sample-500k-384p/ann_train_val/three.json
/content/hagrid_500k/hagrid-sample-500k-384p/ann_train_val/like.json
/content/hagrid_500k/hagrid-sample-500k-384p/ann_train_val/mute.json
/content/hagrid_500k/hagrid-sample-500k-384p/ann_train_val/peace_inverted.json
/content

In [17]:
!pip -q install huggingface_hub hf_transfer pandas pyarrow pillow opencv-python tqdm torch torchvision torchmetrics

In [19]:
import os, glob
candidates = glob.glob("/content/hagrid_500k/**/ann_train_val", recursive=True)
print("ann_train_val folders:", candidates)

DATA_ROOT = os.path.dirname(candidates[0])  # parent of ann_train_val
print("DATA_ROOT =", DATA_ROOT)
print("Contents:", os.listdir(DATA_ROOT)[:20])

ann_train_val folders: ['/content/hagrid_500k/hagrid-sample-500k-384p/ann_train_val']
DATA_ROOT = /content/hagrid_500k/hagrid-sample-500k-384p
Contents: ['train_val_two_up_inverted', 'train_val_three', 'train_val_three2', 'train_val_palm', 'train_val_call', 'train_val_one', 'train_val_peace', 'train_val_four', 'train_val_fist', 'train_val_ok', 'train_val_peace_inverted', 'train_val_like', 'train_val_mute', '.ipynb_checkpoints', 'ann_train_val', 'train_val_stop_inverted', 'train_val_two_up', 'annotations_df.parquet', 'train_val_dislike', 'train_val_stop']


In [20]:
import pandas as pd

df = pd.read_parquet(os.path.join(DATA_ROOT, "annotations_df.parquet"))
print(df.columns)
print(df.head(2))

TARGET = ["palm", "fist", "stop", "stop_inverted"]  # grip not present in this dataset
df = df[df["labels"].apply(lambda xs: any(x in TARGET for x in xs))].reset_index(drop=True)

# Each row may have multiple bboxes/labels; we'll explode to one-hand-per-row below.
print("Rows after filter:", len(df))

Index(['bboxes', 'labels', 'leading_hand', 'leading_conf', 'user_id'], dtype='object')
                                                                                 bboxes  \
00005c9c-3548-4a8f-9d0b-2dd4aff37fc9  [[0.23925175, 0.28595301, 0.25055143, 0.207776...   
0000738b-f640-448f-b697-a6b7d218f5db  [[0.3452406, 0.41348879, 0.16873308, 0.1271769...   

                                                  labels leading_hand  \
00005c9c-3548-4a8f-9d0b-2dd4aff37fc9              [call]        right   
0000738b-f640-448f-b697-a6b7d218f5db  [call, no_gesture]        right   

                                      leading_conf  \
00005c9c-3548-4a8f-9d0b-2dd4aff37fc9           1.0   
0000738b-f640-448f-b697-a6b7d218f5db           1.0   

                                                                                user_id  
00005c9c-3548-4a8f-9d0b-2dd4aff37fc9  5a389ffe1bed6660a59f4586c7d8fe2770785e5bf79b09...  
0000738b-f640-448f-b697-a6b7d218f5db  7bc2b8ae46e76c547837aae519fd5af0389eee

In [23]:
import pandas as pd
import os

parquet_path = os.path.join(DATA_ROOT, "annotations_df.parquet")
df = pd.read_parquet(parquet_path)

print("Columns:")
print(df.columns)

print("\nExample rows:")
print(df.head(2))

Columns:
Index(['bboxes', 'labels', 'leading_hand', 'leading_conf', 'user_id'], dtype='object')

Example rows:
                                                                                 bboxes  \
00005c9c-3548-4a8f-9d0b-2dd4aff37fc9  [[0.23925175, 0.28595301, 0.25055143, 0.207776...   
0000738b-f640-448f-b697-a6b7d218f5db  [[0.3452406, 0.41348879, 0.16873308, 0.1271769...   

                                                  labels leading_hand  \
00005c9c-3548-4a8f-9d0b-2dd4aff37fc9              [call]        right   
0000738b-f640-448f-b697-a6b7d218f5db  [call, no_gesture]        right   

                                      leading_conf  \
00005c9c-3548-4a8f-9d0b-2dd4aff37fc9           1.0   
0000738b-f640-448f-b697-a6b7d218f5db           1.0   

                                                                                user_id  
00005c9c-3548-4a8f-9d0b-2dd4aff37fc9  5a389ffe1bed6660a59f4586c7d8fe2770785e5bf79b09...  
0000738b-f640-448f-b697-a6b7d218f5db  7bc2b8ae46e76c

In [24]:
# Make the index (image id) into a column called "image_id"
df2 = df.reset_index(names="image_id")

TARGET = ["palm", "fist", "stop", "stop_inverted"]  # grip not present in this dataset
df2 = df2[df2["labels"].apply(lambda xs: any(x in TARGET for x in xs))].reset_index(drop=True)

def explode_rows(df_in):
    rows = []
    for _, r in df_in.iterrows():
        image_id = r["image_id"]
        for bbox, label in zip(r["bboxes"], r["labels"]):
            if label in TARGET:
                rows.append({
                    "image_id": image_id,
                    "label": label,
                    "bbox": bbox,
                    "leading_hand": r["leading_hand"],       # optional
                    "leading_conf": float(r["leading_conf"])  # optional
                })
    return pd.DataFrame(rows)

samples_df = explode_rows(df2)

print(samples_df.head())
print("Total samples:", len(samples_df))
print(samples_df["label"].value_counts())

                               image_id label  \
0  0003d89b-0472-4ad4-80f5-96e9969afff5  fist   
1  00046dcf-b4b2-4f63-8a8d-039a1a17b8c8  fist   
2  00086aac-e142-4c40-a7fe-5ad5e60f580b  fist   
3  000b9676-731b-4001-a658-d46b9389fc0e  fist   
4  000ed71d-3661-4aa9-b38b-104d2f3013b4  fist   

                                                bbox leading_hand  \
0  [0.18547158, 0.30972468000000003, 0.17920195, ...        right   
1  [0.05136, 0.568305, 0.27080000000000004, 0.193...        right   
2  [0.48991011, 0.22781381, 0.08205558, 0.0540451...         left   
3  [0.27404238000000003, 0.41426562, 0.08042473, ...        right   
4    [0.34629524, 0.6046897, 0.09789107, 0.07229861]        right   

   leading_conf  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0  
Total samples: 112903
label
stop_inverted    28857
palm             28326
stop             27961
fist             27759
Name: count, dtype: int64


In [25]:
import glob, os

DATA_ROOT = "/content/hagrid_500k/hagrid-sample-500k-384p"  # adjust if yours differs
jpgs = glob.glob(os.path.join(DATA_ROOT, "**", "*.jpg"), recursive=True)
print("Total jpgs found:", len(jpgs))
print("Example jpg paths:", jpgs[:5])

Total jpgs found: 509323
Example jpg paths: ['/content/hagrid_500k/hagrid-sample-500k-384p/train_val_two_up_inverted/ed1004a1-a99e-4fee-a7ac-a98777e5bcd8.jpg', '/content/hagrid_500k/hagrid-sample-500k-384p/train_val_two_up_inverted/7a9efbd5-155c-4461-b36c-27d672f49230.jpg', '/content/hagrid_500k/hagrid-sample-500k-384p/train_val_two_up_inverted/22d2e175-342e-4241-830b-40bead63d035.jpg', '/content/hagrid_500k/hagrid-sample-500k-384p/train_val_two_up_inverted/6ae5aef5-f97b-43be-8155-326e985b5178.jpg', '/content/hagrid_500k/hagrid-sample-500k-384p/train_val_two_up_inverted/5bbc1e74-6112-47cc-b7d3-7bcfe3bf4b4e.jpg']


In [26]:
example_id = samples_df.iloc[0]["image_id"]
hits = [p for p in jpgs if os.path.basename(p) == example_id + ".jpg"]
print("Example id:", example_id)
print("Hits:", hits[:3])

Example id: 0003d89b-0472-4ad4-80f5-96e9969afff5
Hits: ['/content/hagrid_500k/hagrid-sample-500k-384p/train_val_fist/0003d89b-0472-4ad4-80f5-96e9969afff5.jpg']


In [27]:
import pandas as pd
import os

DATA_ROOT = "/content/hagrid_500k/hagrid-sample-500k-384p"  # your root
df = pd.read_parquet(os.path.join(DATA_ROOT, "annotations_df.parquet"))

TARGET = ["palm", "fist", "stop", "stop_inverted"]  # grip not present here

df2 = df.reset_index(names="image_id")
df2 = df2[df2["labels"].apply(lambda xs: any(x in TARGET for x in xs))].reset_index(drop=True)

def explode_rows(df_in):
    rows = []
    for _, r in df_in.iterrows():
        image_id = r["image_id"]
        for bbox, label in zip(r["bboxes"], r["labels"]):
            if label in TARGET:
                rows.append({
                    "image_id": image_id,
                    "label": label,
                    "bbox": bbox,
                })
    return pd.DataFrame(rows)

samples_df = explode_rows(df2)
print("Total crop samples:", len(samples_df))
print(samples_df["label"].value_counts())

Total crop samples: 112903
label
stop_inverted    28857
palm             28326
stop             27961
fist             27759
Name: count, dtype: int64


In [28]:
import glob, os

all_imgs = glob.glob(os.path.join(DATA_ROOT, "**", "*.jpg"), recursive=True)
print("Total jpgs:", len(all_imgs))

img_map = {os.path.splitext(os.path.basename(p))[0]: p for p in all_imgs}

# sanity check
ex_id = samples_df.iloc[0]["image_id"]
print("Example id:", ex_id)
print("Resolved path:", img_map.get(ex_id))

Total jpgs: 509323
Example id: 0003d89b-0472-4ad4-80f5-96e9969afff5
Resolved path: /content/hagrid_500k/hagrid-sample-500k-384p/train_val_fist/0003d89b-0472-4ad4-80f5-96e9969afff5.jpg


In [29]:
!pip -q install scikit-learn
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    samples_df,
    test_size=0.1,
    random_state=42,
    stratify=samples_df["label"]
)

print(len(train_df), len(val_df))

101612 11291


In [30]:
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class_to_idx = {c:i for i,c in enumerate(TARGET)}

class HagridCropDataset(Dataset):
    def __init__(self, df, img_map, class_to_idx, transform=None, pad=0.15):
        self.df = df.reset_index(drop=True)
        self.img_map = img_map
        self.class_to_idx = class_to_idx
        self.transform = transform
        self.pad = pad

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        r = self.df.iloc[idx]
        image_id = r["image_id"]
        label = r["label"]
        x, y, w, h = r["bbox"]  # normalized [0..1]

        path = self.img_map[image_id]
        img_bgr = cv2.imread(path)
        H, W = img_bgr.shape[:2]

        # normalized -> pixels
        x1 = int(x * W); y1 = int(y * H)
        x2 = int((x + w) * W); y2 = int((y + h) * H)

        # padding
        pad_px = int(self.pad * max(x2 - x1, y2 - y1))
        x1 = max(0, x1 - pad_px); y1 = max(0, y1 - pad_px)
        x2 = min(W, x2 + pad_px); y2 = min(H, y2 + pad_px)

        crop_bgr = img_bgr[y1:y2, x1:x2]
        crop_rgb = cv2.cvtColor(crop_bgr, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            x_t = self.transform(crop_rgb)
        else:
            x_t = torch.from_numpy(crop_rgb).permute(2,0,1).float() / 255.0

        y_t = self.class_to_idx[label]
        return x_t, y_t

IMG_SIZE = 224
train_tfms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ColorJitter(0.2, 0.2, 0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
val_tfms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

train_ds = HagridCropDataset(train_df, img_map, class_to_idx, transform=train_tfms)
val_ds   = HagridCropDataset(val_df,   img_map, class_to_idx, transform=val_tfms)

train_dl = DataLoader(train_ds, batch_size=64, shuffle=True,  num_workers=2, pin_memory=True)
val_dl   = DataLoader(val_ds,   batch_size=64, shuffle=False, num_workers=2, pin_memory=True)

len(train_ds), len(val_ds)


(101612, 11291)

In [31]:
import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights
from torchmetrics.classification import MulticlassAccuracy
from tqdm.auto import tqdm
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = resnet18(weights=ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, len(TARGET))
model = model.to(device)

loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

acc = MulticlassAccuracy(num_classes=len(TARGET)).to(device)

def run_epoch(dl, train=True):
    model.train() if train else model.eval()
    total_loss = 0.0
    acc.reset()

    ctx = torch.enable_grad() if train else torch.no_grad()
    with ctx:
        for x, y in tqdm(dl, leave=False):
            x = x.to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)

            logits = model(x)
            loss = loss_fn(logits, y)

            if train:
                opt.zero_grad()
                loss.backward()
                opt.step()

            total_loss += loss.item() * x.size(0)
            acc.update(logits.softmax(dim=1), y)

    return total_loss / len(dl.dataset), acc.compute().item()

for epoch in range(1, 6):
    tr_loss, tr_acc = run_epoch(train_dl, train=True)
    va_loss, va_acc = run_epoch(val_dl,   train=False)
    print(f"Epoch {epoch}: train loss {tr_loss:.4f} acc {tr_acc:.3f} | val loss {va_loss:.4f} acc {va_acc:.3f}")

  0%|          | 0/1588 [00:00<?, ?it/s]

  0%|          | 0/177 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7eb71e5dd3a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
    if w.is_alive():
       ^Exception ignored in: ^<function _MultiProcessingDataLoaderIter.__del__ at 0x7eb71e5dd3a0>^
Traceback (most recent call last):
^  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
^    self._shutdown_workers()^
^^^^^  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    if w.is_alive():    
assert self._parent_pid == os.getpid(), 'can only test a child process'
              ^ ^^  ^ ^^^^^^^^^^^^^^
^  Fil

Epoch 1: train loss 0.0419 acc 0.985 | val loss 0.0148 acc 0.996


  0%|          | 0/1588 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7eb71e5dd3a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7eb71e5dd3a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 16

KeyboardInterrupt: 

In [32]:
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True,  num_workers=0, pin_memory=True, persistent_workers=False)
val_dl   = DataLoader(val_ds,   batch_size=64, shuffle=False, num_workers=0, pin_memory=True, persistent_workers=False)


In [33]:
import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights
from torchmetrics.classification import MulticlassAccuracy
from tqdm.auto import tqdm
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = resnet18(weights=ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, len(TARGET))
model = model.to(device)

loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

acc = MulticlassAccuracy(num_classes=len(TARGET)).to(device)

def run_epoch(dl, train=True):
    model.train() if train else model.eval()
    total_loss = 0.0
    acc.reset()

    ctx = torch.enable_grad() if train else torch.no_grad()
    with ctx:
        for x, y in tqdm(dl, leave=False):
            x = x.to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)

            logits = model(x)
            loss = loss_fn(logits, y)

            if train:
                opt.zero_grad()
                loss.backward()
                opt.step()

            total_loss += loss.item() * x.size(0)
            acc.update(logits.softmax(dim=1), y)

    return total_loss / len(dl.dataset), acc.compute().item()

for epoch in range(1, 6):
    tr_loss, tr_acc = run_epoch(train_dl, train=True)
    va_loss, va_acc = run_epoch(val_dl,   train=False)
    print(f"Epoch {epoch}: train loss {tr_loss:.4f} acc {tr_acc:.3f} | val loss {va_loss:.4f} acc {va_acc:.3f}")

  0%|          | 0/1588 [00:00<?, ?it/s]

  0%|          | 0/177 [00:00<?, ?it/s]

Epoch 1: train loss 0.0420 acc 0.985 | val loss 0.0149 acc 0.995


  0%|          | 0/1588 [00:00<?, ?it/s]

  0%|          | 0/177 [00:00<?, ?it/s]

...

  0%|          | 0/177 [00:00<?, ?it/s]

Epoch 4: train loss 0.0105 acc 0.997 | val loss 0.0185 acc 0.995


  0%|          | 0/1588 [00:00<?, ?it/s]

  0%|          | 0/177 [00:00<?, ?it/s]

Epoch 5: train loss 0.0097 acc 0.997 | val loss 0.0187 acc 0.994


In [34]:
torch.save({
    "state_dict": model.state_dict(),
    "classes": TARGET,
    "img_size": IMG_SIZE
}, "/content/gesture_resnet18.pt")

In [35]:
from google.colab import files
files.download("/content/gesture_resnet18.pt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>